In [1]:
from google.colab import drive
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
from keras.preprocessing.image import ImageDataGenerator

# Chelsea's Probs
train_dir = "/gdrive/My Drive/deep_learning/EGD"

img_width = 224
img_height = 224
batch_size = 1
channels = 3

train_datagen = ImageDataGenerator(rescale=1./255)                           

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')   

nb_train_samples = len(train_generator.filenames)

Using TensorFlow backend.


Found 102 images belonging to 2 classes.


In [7]:
#Dont touch!

import numpy as np

def stacking_samples(dataset_type, sample_count):
    features = np.zeros(shape=(sample_count, img_height, img_width, channels))
    labels = np.zeros(shape=(sample_count))

    i = 0
    if dataset_type == "train":
        for inputs_batch, labels_batch in train_generator:
            features[i * batch_size : (i + 1) * batch_size] = inputs_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            print(i * batch_size)
            if i * batch_size >= sample_count:
                break 
    return features, labels


train_features, train_labels = stacking_samples("train", nb_train_samples)
print(train_features.shape, train_labels.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (171407884 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
(102, 224, 224, 3) (102,)


In [8]:
from keras import backend as K
import tensorflow as tf
from PIL import Image
from keras.applications import VGG16

Image.MAX_IMAGE_PIXELS = None

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_height, img_width, channels))

#Extracting features from OCT data using pretrained VGG
def extract_features(raw_x, raw_y):
    sample_count = len(raw_x)
    
    features = conv_base.predict(raw_x)
    labels = raw_y
    
    return features, labels

58892288/58889256 [==============================] - 5s 0us/step


In [0]:
def rf_training(x_train_f, y_train_f, x_val_f, y_val_f, score_list):
  clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
  fitter = clf.fit(np.reshape(x_train_f, (len(x_train_f), 7*7*512)), y_train_f)
  
  #validation
#   print(x_val_f.shape)
  valid_prediction = clf.predict(np.reshape(x_val_f, (len(x_val_f), 7*7*512)))
  valid_score = clf.score(np.reshape(x_val_f, (len(x_val_f), 7*7*512)), y_val_f)
  score_list.append(valid_score)
  
#   print("valid_prediction = ", valid_prediction)
#   print("ground_truth", y_val_f)
  
  return fitter, x_val_f, valid_prediction
  

# ROC

In [0]:
import sklearn
from sklearn.metrics import roc_curve, auc

def prepare_ROC_binary(fitter, x_val_f, y_val_f):
  
  n_classes = 2

  proba = fitter.predict_proba(np.reshape(x_val_f, (len(x_val_f), 7*7*512)))

  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], thresholds = roc_curve(y_val_f, proba[:, i], pos_label=i)
#       print("fpr, tpr", fpr[i], tpr[i])
      roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])
#       print("test AUC for class", i, roc_auc[i])
#       print("thresholds", thresholds)
  
  return fpr, tpr, roc_auc
      

In [0]:
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle

def saveROCfigure(fpr, tpr, roc_auc, ith):
  
  n_classes = 2
  
  lw = 2

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

#   print(all_fpr)

  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(n_classes):
      mean_tpr += interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= n_classes

  colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
  for i, color in zip(range(n_classes), colors):
      plt.plot(fpr[i], tpr[i], color=color, lw=lw,
               label='ROC curve of class {0} (area = {1:0.2f})'
               ''.format(i, roc_auc[i]))

  plt.plot([0, 1], [0, 1], 'k--', lw=lw)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Some extension of Receiver operating characteristic to multi-class')
  plt.legend(loc="lower right")
  plt.savefig("vgg16_rf_egd_" + str(ith) + ".png")
  plt.clf()
#   plt.show()

# 10-fold

In [12]:
#10 folds

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics


folds = 10

kf = KFold(n_splits=folds, shuffle=True)

accs = []
i = 0
for train_index, val_index in kf.split(train_features):
  
  #collect data
  x_train = train_features[train_index]
  y_train = train_labels[train_index]
  x_val = train_features[val_index]
  y_val = train_labels[val_index]
  
  #feature extraction on the training set
  x_train_f, y_train_f = extract_features(x_train, y_train)
  # and on the validation set
  x_val_f, y_val_f = extract_features(x_val, y_val)
  
  #rf on the result
  fitter, x_val, y_val = rf_training(x_train_f, y_train_f, x_val_f, y_val_f, accs)
  fpr, tpr, auc = prepare_ROC_binary(fitter, x_val_f, y_val_f)
  saveROCfigure(fpr, tpr, auc, i)
  print("acc = ", accs[i])
  i += 1
  
print("accs", accs)
print("mean acc =", np.mean(accs))

acc =  0.9090909090909091
acc =  1.0
acc =  1.0
acc =  1.0
acc =  0.8
acc =  0.9
acc =  0.9
acc =  1.0
acc =  1.0
acc =  0.9
accs [0.9090909090909091, 1.0, 1.0, 1.0, 0.8, 0.9, 0.9, 1.0, 1.0, 0.9]
mean acc = 0.940909090909091
